<a href="https://colab.research.google.com/github/iakob12345/test/blob/master/Wine_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Introduction**

##1.1.Importing Libraries

In [ ]:
!pip install pycountry

     |████████████████████████████████| 10.1MB 5.9MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=5a9c395e0f2e31ae6634390ff1edd7efb53bead12fe324282035f5524121ceb8
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry


In [ ]:
%matplotlib inline
from scipy import stats
import pandas as pd
import seaborn as sns
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from google.colab import drive
from plotly import graph_objs as go
import plotly.figure_factory as ff
import pycountry

drive.mount('/content/drive/')

path='/content/drive/My Drive/Colab Notebooks/Wine_dataset.csv'

# read cvs in Pandas
winedata_df = pd.read_csv(path)

Mounted at /content/drive/


## Data Introduction

In [ ]:
winedata_df.head(5)

,catalog,wineries,wine_names,countries,regions,prices,rating_avgs,total_reviews,link_wines
0,white_usa,Charles Smith,Kung Fu Girl Riesling 2017,Vereinigte Staaten,Ancient Lakes of Columbia Valley,16.92,3.8,5585,https://www.vivino.com/charles-smith-kung-fu-g...
1,white_usa,Charles Smith,Kung Fu Girl Riesling 2015,Vereinigte Staaten,Ancient Lakes of Columbia Valley,16.92,3.8,4555,https://www.vivino.com/charles-smith-kung-fu-g...
2,white_usa,Chateau Ste. Michelle,Riesling 2015,Vereinigte Staaten,Columbia Valley,10.72,3.8,3856,https://www.vivino.com/chateau-ste-michelle-ri...
3,white_usa,Kendall-Jackson,Vintner's Reserve Chardonnay 2017,Vereinigte Staaten,California,20.70,3.8,3379,https://www.vivino.com/kendall-jackson-vintner...
4,white_usa,Wente,Morning Fog Chardonnay 2016,Vereinigte Staaten,Livermore Valley,14.59,3.9,3307,https://www.vivino.com/wente-vineyards-morning...


In [ ]:
winedata_df.shape

(27745, 9)

In [ ]:
#catalog is a weak, user-inputted dimension, which will be redefined below by stronger datapoints
winedata_df.groupby(by='catalog')['link_wines'].agg(count='count').reset_index().sort_values(by='count',ascending=False).head(10)

,catalog,count
1,Rose,1595
28,sparkling_spain,1104
3,red_australia,975
17,red_spain_high,950
13,red_italy_low,925
24,sparkling_france,925
7,red_france_low,925
41,white_italy_medium_high,923
33,white_austria,900
10,red_france_vip,900


# **2.Data validation**

Before we proceed, first we want to check if the scraper actually performed its job as intended and collected the correct information. For this, we will take **stratified data samples** based on price groups and then manually cross-check the data between the scraped file and the website.


In [ ]:
pd.set_option('display.max_colwidth', -1)

#creating a list of dictionaries where the key is price and the value is link

stratum_popular = winedata_df[winedata_df['prices'] <= 10]
stratum_popular_premium = winedata_df[(winedata_df['prices'] <= 50) & (winedata_df['prices'] > 10)]
stratum_premium = winedata_df[(winedata_df['prices'] <= 100) & (winedata_df['prices'] > 50)]
stratum_luxury = winedata_df[(winedata_df['prices'] <= 500) & (winedata_df['prices'] > 100)]
stratum_super_luxury = winedata_df[(winedata_df['prices'] <= 5000) & (winedata_df['prices'] > 500)]
stratum_icon = winedata_df[winedata_df['prices'] > 5000]
list_stratums = [stratum_popular,stratum_popular_premium,stratum_premium,stratum_luxury,stratum_super_luxury,stratum_icon]

stratified_sampling_list = []

for stratum in list_stratums:
  stratum = stratum[['prices','countries','rating_avgs','total_reviews','link_wines']].sample(10,random_state=0)
  stratified_sampling_list.append(stratum)

stratified_sampling_df = pd.concat(stratified_sampling_list)
stratified_sampling_df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



,prices,countries,rating_avgs,total_reviews,link_wines
13386,8.45,Österreich,3.7,72,https://www.vivino.com/weingut-fred-loimer-rose/w/1654412?year=2018&price_id=20696080
13065,7.95,Italien,3.7,251,https://www.vivino.com/collefrisio-rose-vino-da-tavola/w/4437725?year=N.V.&price_id=19403690
14125,8.85,Spanien,3.9,634,https://www.vivino.com/borsao-crianza-seleccion-crianza/w/1289492?year=2014&price_id=20134405
466,9.59,Vereinigte Staaten,3.5,31,https://www.vivino.com/crane-lake-chardonnay/w/87405?year=2018&price_id=20737673
5614,6.50,Frankreich,3.6,397,https://www.vivino.com/les-jamelles-viognier/w/10552?year=2017&price_id=17244555


Right after the scraper finished its job, we doublechecked these sample elements by hand and all **wine catalogs on the website showed the same information as on the collected file**. Therefore, we concluded that scraper did its job properly and brought the right information. 

Please note that reviews,ratings and prices change slightly over time and when you view this file it might be different. This analysis is based on the information as of 25th of April, 2020.

# **3.Data Tidying**

##3.1.Extracting Wine Type

The first column 'catalog' contains a concatenated value of **wine type** with other parameters. I will extract wine type and store it as a separate column since this is a crucial information for the further analysis.

In [ ]:
#the first column contains concatenation values of wine type and country. I will extract the 'Wine type' from this column

def extract_wine_type(element):
  element = str(element)
  element = element.split("_")
  element = element[0]
  return element
  
winedata_df['wine_type'] = winedata_df['catalog'].map(extract_wine_type)
winedata_df['wine_type'] = winedata_df['wine_type'].str.lower()

print(winedata_df['wine_type'].value_counts(dropna=False))

red          13231
white        8668 
sparkling    3501 
rose         1595 
dessert      750  
Name: wine_type, dtype: int64


##3.2.Mapping price to price buckets

Next, mapping individual wine prices to price buckets:
  
*   Less than 10 euros --> 'Low'
*   Between 10 and 25 --> 'Medium'
*   Between 25 and 100 --> 'Premium'
*   More than 100 --> 'Luxury'  

In [ ]:
def price_buckets(element):
  if element <= 10:
    return "Low"
  elif element>10 and element<=25:
    return "Medium"
  elif element>25 and element<=100:
    return "Premium"
  elif element>100:
    return "Luxury"

winedata_df['price_category'] = winedata_df['prices'].map(price_buckets)
winedata_df['price_category'].value_counts()

Medium     11037
Low        7344 
Premium    7067 
Luxury     1873 
Name: price_category, dtype: int64

##3.3.Deriving 'Value' column

Next, deriving **'value'** column by dividing Rating to Price and multiply to 100 as defined in the introduction.

In [ ]:
def value_calculation(column1,column2):
  value = ((column1/column2) * 100).round(2)
  return value

winedata_df['value'] =  value_calculation(winedata_df['rating_avgs'],winedata_df['prices'])

## 3.7.Remapping country names: from German to English + country iso_3

Lastly, I will **translate** the country names from German to English to be easier for non-German speakers. Also, I will map country names to iso_3 format, so we can use it later in Plotly

In [ ]:
countries_de_to_en = {'Frankreich':'France','Portugal':'Portugal','Chile':'Chile','Italien':'Italy','Spanien':'Spain','Deutschland':'Germany','Österreich':'Austria','Australien':'Australia','Vereinigte Staaten':'United States','Argentinien':'Argentina','Südafrika':'South Africa'}
winedata_df['countries'] = winedata_df['countries'].map(countries_de_to_en)
winedata_df['countries'] = winedata_df['countries'].astype(str)
winedata_df['country_iso3'] = winedata_df['countries'].map(lambda x: pycountry.countries.get(name=x).alpha_3 if x != 'nan' else x)

winedata_df['countries'].value_counts(dropna=False)


France           8123
Italy            5849
Spain            3743
Germany          2576
Austria          1793
Portugal         1541
Australia        1383
Chile            1146
United States    745 
Argentina        611 
South Africa     127 
nan              108 
Name: countries, dtype: int64

## 3.4.Deriving a 'Category' and 'Group' column

Finally, I will create a category column that will concatenate country, price_category and wine_type - since this is our desired grouping


In [ ]:
winedata_df['group'] = [str(x).capitalize() + '-' + str(y).capitalize() + '-' + str(z).capitalize() for x, y, z in zip(winedata_df['countries'], winedata_df['wine_type'],winedata_df['price_category'])]
winedata_df['category'] = winedata_df.apply(lambda x: str(x.wine_type).capitalize() + '-' + str(x.price_category).capitalize(), axis=1)

winedata_df['category'].value_counts()

Red-Medium           4830
White-Medium         3897
Red-Premium          3553
Red-Low              3321
White-Low            2182
White-Premium        2159
Sparkling-Medium     1555
Red-Luxury           1311
Rose-Low             947 
Sparkling-Premium    918 
Sparkling-Low        847 
Rose-Medium          541 
Dessert-Premium      354 
White-Luxury         338 
Red-None             216 
Dessert-Medium       214 
Sparkling-Luxury     165 
White-None           92  
Rose-Premium         83  
Dessert-None         79  
Dessert-Luxury       56  
Dessert-Low          47  
Rose-None            21  
Sparkling-None       16  
Rose-Luxury          3   
Name: category, dtype: int64

## 3.5.Deriving 'Harvest year' and 'Grape Type'

In [ ]:
def grape_year(grape):
  grape = str(grape)
  grape_split = grape.split(" ")
  year = grape_split[-1]
  if any(char.isdigit() for char in year):
    return year
  else:
    return "No Year Detected"

winedata_df['harvest_year'] = winedata_df['wine_names'].map(grape_year)

winedata_df['harvest_year'].value_counts().head(10)

2016                4171
2017                3996
2018                3802
2015                3586
No Year Detected    3576
2014                2096
2013                1440
2012                1084
2011                838 
2010                532 
Name: harvest_year, dtype: int64

In [ ]:
def grape_type(grape):
  grape = str(grape)
  grape = grape.rsplit(' ', 1)[0]
  grape = str(grape)
  return grape
  
winedata_df['grape'] = winedata_df['wine_names'].map(grape_type)
winedata_df['grape'].value_counts().head(10)

Chardonnay                                            330
Pomerol                                               203
Cabernet Sauvignon                                    168
Saint-Émilion Grand Cru (Premier Grand Cru Classé)    156
Saint-Julien (Grand Cru Classé)                       153
Sauternes (Premier Grand Cru Classé)                  136
Pauillac (Grand Cru Classé)                           134
Haut-Médoc                                            133
Sauvignon Blanc                                       133
Tinto                                                 122
Name: grape, dtype: int64

##3.6.Renaming columns and adjusting their positions

Next, **rearranging column names** and their positions for a better readability

In [ ]:
mapping={'catalog':'catalog','wineries':'winery','grape':'grape','harvest_year': 'harvest_year','category':'category','price_category':'price_category','countries':'country','regions':'region','prices':'price','rating_avgs':'rating_avg','total_reviews':'total_reviews','link_wines':'link','value':'value','wine_type':'wine_type'}
winedata_df  = winedata_df.rename(mapping,axis=1)

#mapping = {''}
print(winedata_df.columns.values)



['catalog' 'winery' 'wine_names' 'country' 'region' 'price' 'rating_avg'
 'total_reviews' 'link' 'wine_type' 'price_category' 'value'
 'country_iso3' 'group' 'category' 'harvest_year' 'grape']


In [ ]:
#repositioning columns
winedata_df = winedata_df[['catalog','wine_type','winery','grape','harvest_year','country','region','price','price_category','rating_avg','total_reviews','value','category','link','country_iso3','group']]


##Defining useful functions

In [ ]:
def show_null_values():
  null_columns=winedata_df.columns[winedata_df.isnull().any()]
  results = winedata_df[null_columns].isnull().sum()
  if results.sum() != 0.0:
    print('Null values Exloration per column')
    print(results)
    print('Total Nr of Null values:')
    print(results.sum())
  else:
    print('Total Nr of null values found: 0')

In [ ]:
def show_snapshot_values(column_of_value_counts=False, is_asc=True, results_to_display = 5):
  duplicates_sum = winedata_df.duplicated('link').sum()
  print('Nr of rows:', winedata_df.shape[0])   
  print('Nr of columns:',winedata_df.shape[1])
  print('_____________________________________')
  print(' ')
  show_null_values()
  print('_____________________________________')
  print(' ')
  print('Total Nr of Duplicates found:',str(duplicates_sum))
  if column_of_value_counts is not False:
    print('_____________________________________')
    print(' ')
    value_counts = winedata_df[column_of_value_counts].value_counts(ascending=is_asc).head(results_to_display)
    print(f'   Value Counts for {column_of_value_counts}')
    print(value_counts)
  else:
    print('')

# **4.Clearing Duplicates**

##Exploration of duplicates

After tidying the data, duplicates will be addressed. 
The duplications are found by rows having the same wine link, which should be unique per wine.


In [ ]:
show_snapshot_values()

Nr of rows: 27745
Nr of columns: 16
_____________________________________
 
Null values Exloration per column
winery            3  
price             424
price_category    424
value             424
dtype: int64
Total Nr of Null values:
1275
_____________________________________
 
Total Nr of Duplicates found: 2050



## Dropping Duplicates

In [ ]:
winedata_df = winedata_df.sort_values(['price','country'],ascending=True)
winedata_df = winedata_df.drop_duplicates(subset=['link'],keep='first')

In [ ]:
show_snapshot_values()

Nr of rows: 25695
Nr of columns: 16
_____________________________________
 
Null values Exloration per column
winery            3  
price             400
price_category    400
value             400
dtype: int64
Total Nr of Null values:
1203
_____________________________________
 
Total Nr of Duplicates found: 0



# **5.Solving Missing Values**

## Exploration

### Undesirable 0 values

In [ ]:
print(winedata_df.describe())

              price    rating_avg  total_reviews         value
count  25295.000000  25695.000000  25695.000000   25295.000000
mean   52.357881     3.879887      588.200856     25.578572   
std    308.213608    0.323020      1759.082588    17.166072   
min    1.850000      2.000000      25.000000      0.020000    
25%    9.945000      3.700000      81.000000      11.610000   
50%    16.730000     3.900000      190.000000     23.080000   
75%    34.900000     4.100000      541.000000     36.700000   
max    22800.000000  4.800000      91586.000000   183.780000  


In [ ]:
print((winedata_df == '0' ).sum())

catalog           0
wine_type         0
winery            0
grape             0
harvest_year      0
country           0
region            0
price             0
price_category    0
rating_avg        0
total_reviews     0
value             0
category          0
link              0
country_iso3      0
group             0
dtype: int64


Above, we see that none of the columns have 0 values

### Null values

Below, we Explore **count of Null values** per column

In [ ]:
show_snapshot_values()

Nr of rows: 25695
Nr of columns: 16
_____________________________________
 
Null values Exloration per column
winery            3  
price             400
price_category    400
value             400
dtype: int64
Total Nr of Null values:
1203
_____________________________________
 
Total Nr of Duplicates found: 0



## Action points

### Replacing or dropping rows with missing prices?



The most valuable columns for research are **'Price'**, **'Price_category'** and **'Value'**. The latter 2 columns are derived from the 'Price', therefore fixing the 'Price' will eventually have a positive cascading effect to the rest of the columns.

For fixing the missing values of price, I will explore the possibility of **replacing** its values by the **average price** of the categories they belong it. However, before doing so, I would like to check the **relative standard deviation (*Coefficient of Variation (CV)*)** of each category in relation to price:
*   If category's CV is low *(<= 0.5)*, I will **replace** missing values with average prices per category
*   If category's CV's high *(> 0.5)*, I will **drop** associated wines



In [ ]:
#finding catalogs that have null prices
is_price_null_rows = winedata_df['price'].isnull()
null_price_catalog_rows = winedata_df[is_price_null_rows]['catalog']
unique_null_catalogs = list(pd.unique(null_price_catalog_rows))

#building pivot table based on null_catalog values
df_null_catalogs = winedata_df[winedata_df['catalog'].isin(unique_null_catalogs)]
pivot_null_catalogs = pd.pivot_table(df_null_catalogs,index='catalog',values = 'price',aggfunc=['mean',np.std,'count']).round(0)

#converting to dataframe for later uses
null_cat_df = pd.DataFrame(pivot_null_catalogs.to_records())

#renaming original pivot columns
mapping_temp = {"catalog":"catalog","('mean', 'price')":"mean_price","('std', 'price')":"std_price","('count', 'price')":"count_wines"}
null_cat_df = null_cat_df.rename(mapping_temp,axis=1)

#introducing CV
null_cat_df['CV'] = null_cat_df['std_price']/null_cat_df['mean_price']

null_cat_df.sort_values(by='count_wines',ascending=False).head(10)



,catalog,mean_price,std_price,count_wines,CV
3,red_australia,50.0,72.0,961,1.440000
1,Rose,12.0,10.0,950,0.833333
15,red_spain_high,48.0,63.0,923,1.312500
6,red_france_low,8.0,4.0,922,0.500000
18,sparkling_france,66.0,75.0,920,1.136364
12,red_italy_low,8.0,2.0,910,0.250000
27,white_italy_medium_high,29.0,26.0,897,0.896552
20,white_austria,25.0,21.0,883,0.840000
5,red_chile,25.0,35.0,869,1.400000
8,red_france_veryhigh,75.0,37.0,837,0.493333


In [ ]:
#dividing catalogs with high and low CV
low_cv_catalog = null_cat_df[null_cat_df['CV'] <= 0.5]['catalog'].reset_index(drop=True)
high_cv_catalog = null_cat_df[null_cat_df['CV'] > 0.5]['catalog'].reset_index(drop=True)

null_cat_df[null_cat_df['CV'] <= 0.5].shape

(11, 5)

### WInes with Low Coefficient of Variation - Replacing missing wine prices by average prices

In [ ]:
show_snapshot_values()

Nr of rows: 25695
Nr of columns: 16
_____________________________________
 
Null values Exloration per column
winery            3  
price             400
price_category    400
value             400
dtype: int64
Total Nr of Null values:
1203
_____________________________________
 
Total Nr of Duplicates found: 0



In [ ]:
#getting a dictionary from pivot table where key is catalog and value - mean_price in order to iterate afterwords
low_cv_df = null_cat_df[null_cat_df['CV'] <= 0.5]
low_cv_df_catalog_mean_price = low_cv_df[['catalog','mean_price']]


#filtering original dataframe for null_priced and low_cv_catalog rows
filter_null_price_low_cv = is_price_null_rows & winedata_df['catalog'].isin(low_cv_catalog)

#merging the mean price dataframe to the original one so then I can replace nan values in the Price with mean_price
winedata_df = winedata_df.merge(low_cv_df_catalog_mean_price, on='catalog', how ='left')

def null_to_mean_price(first_column, second_column):
  if np.isnan(first_column) and pd.notnull(second_column):
    return second_column
  else:
    return first_column

winedata_df['price'] = winedata_df['price'].combine(winedata_df['mean_price'],null_to_mean_price)



#### Results of the Low CV operation

In [ ]:
show_snapshot_values()

Nr of rows: 25695
Nr of columns: 17
_____________________________________
 
Null values Exloration per column
winery            3    
price             337  
price_category    400  
value             400  
mean_price        17313
dtype: int64
Total Nr of Null values:
18453
_____________________________________
 
Total Nr of Duplicates found: 0



####Recalculating columns derived from 'Price' which are 'price_category', 'value', 'group'

Since we have now more prices available, we need to recalculate derived columns from it

In [ ]:
winedata_df['price_category'] = winedata_df['price'].map(price_buckets)
winedata_df['value'] = value_calculation(winedata_df['rating_avg'],winedata_df['price'])
winedata_df['category'] = winedata_df.apply(lambda x: str(x.wine_type).capitalize() + '-' + str(x.price_category).capitalize(), axis=1)

show_snapshot_values()

#note: mean_price is an intermediary column nad will be dropped

Nr of rows: 25695
Nr of columns: 17
_____________________________________
 
Null values Exloration per column
winery            3    
price             337  
price_category    337  
value             337  
mean_price        17313
dtype: int64
Total Nr of Null values:
18327
_____________________________________
 
Total Nr of Duplicates found: 0



###Wines with high Coefficient of Variation - Dropping missing wine prices

In [ ]:
high_cv_catalog = null_cat_df[null_cat_df['CV'] > 0.5]
high_cv_catalog = high_cv_catalog['catalog']
high_cv_catalog = list(high_cv_catalog)

winedata_df['high_cv_catalog'] = winedata_df['catalog'].map(lambda x: True if x in(high_cv_catalog) else False)
winedata_df['null_priced_high_cv_catalog_wines'] = winedata_df['high_cv_catalog'] & winedata_df['price'].isnull()

In [ ]:
#dropping null_priced_high_cv_catalog_wines
winedata_df.drop(index=winedata_df[winedata_df['null_priced_high_cv_catalog_wines']].index,inplace=True)
#dropping rows with null 'winery' values
winedata_df.drop(index=winedata_df[winedata_df['winery'].isnull()].index,inplace=True)
#dropping intermediary mean_price
winedata_df.drop(columns='mean_price',axis=0,inplace=True)

### Results of both operations on Missing Values


In [ ]:
show_snapshot_values()

Nr of rows: 25355
Nr of columns: 18
_____________________________________
 
Total Nr of null values found: 0
_____________________________________
 
Total Nr of Duplicates found: 0



In [ ]:
winedata_df.head()

,catalog,wine_type,winery,grape,harvest_year,country,region,price,price_category,rating_avg,total_reviews,value,category,link,country_iso3,group,high_cv_catalog,null_priced_high_cv_catalog_wines
0,red_portugal,red,Bacalhôa,Alentejano Monte das Ânforas Tinto,2014,Portugal,Alentejano,1.85,Low,3.4,740,183.78,Red-Low,https://www.vivino.com/bacalhoa-vinhos-de-portugal-alentejano-monte-das-anforas-tinto/w/1706071?year=2014&price_id=16543955,PRT,Portugal-Red-Low,True,False
1,red_portugal,red,AR - Adega de Redondo,Porta Da Ravessa Alentejo Tinto,2017,Portugal,Alentejo,2.39,Low,3.4,1016,142.26,Red-Low,https://www.vivino.com/adega-cooperativa-de-redondo-porta-da-ravessa-alentejo-tinto/w/1301087?year=2017&price_id=18902719,PRT,Portugal-Red-Low,True,False
2,red_portugal,red,AR - Adega de Redondo,Porta Da Ravessa Alentejo Tinto,2018,Portugal,Alentejo,2.39,Low,3.4,479,142.26,Red-Low,https://www.vivino.com/adega-cooperativa-de-redondo-porta-da-ravessa-alentejo-tinto/w/1301087?year=2018&price_id=18902720,PRT,Portugal-Red-Low,True,False
3,Rose,rose,Canals & Nubiola,Rosado,No Year Detected,Spain,Catalunya,2.65,Low,2.9,360,109.43,Rose-Low,https://www.vivino.com/canals-and-nubiola-rosado/w/2208190?year=N.V.&price_id=7583293,ESP,Spain-Rose-Low,True,False
4,red_portugal,red,Alfacinha,Tinto,2018,Portugal,Lisboa,2.80,Low,3.5,385,125.00,Red-Low,https://www.vivino.com/alfacinha-tinto/w/6877515?year=2018&price_id=21308016,PRT,Portugal-Red-Low,True,False


## Retidying columns after Null related operations

Dropping some columns since they will not be needed anymore.

In [ ]:
winedata_df.columns

Index(['catalog', 'wine_type', 'winery', 'grape', 'harvest_year', 'country',
       'region', 'price', 'price_category', 'rating_avg', 'total_reviews',
       'value', 'category', 'link', 'country_iso3', 'group', 'high_cv_catalog',
       'null_priced_high_cv_catalog_wines'],
      dtype='object')

In [ ]:
winedata_df.drop(axis=1,columns=['high_cv_catalog', 'null_priced_high_cv_catalog_wines'],inplace=True)

# **6.Cleaning Data by Representation**

##Dropping Categories little representation (<100 bottles)

Population ('category') that has less than 10 wines will be dropped

In [ ]:
show_snapshot_values('category',True,10)

Nr of rows: 25355
Nr of columns: 16
_____________________________________
 
Total Nr of null values found: 0
_____________________________________
 
Total Nr of Duplicates found: 0
_____________________________________
 
   Value Counts for category
Rose-Luxury         2  
Dessert-Low         47 
Rose-Premium        50 
Dessert-Luxury      56 
Sparkling-Luxury    159
Dessert-Medium      214
White-Luxury        314
Rose-Medium         319
Dessert-Premium     354
Rose-Low            578
Name: category, dtype: int64


In [ ]:
winedata_df = winedata_df.groupby('category').filter(lambda x : len(x)>100)

In [ ]:
show_snapshot_values('category',True,10)

Nr of rows: 25200
Nr of columns: 16
_____________________________________
 
Total Nr of null values found: 0
_____________________________________
 
Total Nr of Duplicates found: 0
_____________________________________
 
   Value Counts for category
Sparkling-Luxury     159 
Dessert-Medium       214 
White-Luxury         314 
Rose-Medium          319 
Dessert-Premium      354 
Rose-Low             578 
Sparkling-Low        590 
Sparkling-Premium    819 
Sparkling-Medium     1109
Red-Luxury           1309
Name: category, dtype: int64


##Dropping Groups with little representation (<50 bottles)


In [ ]:
show_snapshot_values('group',True,10)

Nr of rows: 25200
Nr of columns: 16
_____________________________________
 
Total Nr of null values found: 0
_____________________________________
 
Total Nr of Duplicates found: 0
_____________________________________
 
   Value Counts for group
Chile-White-Luxury               1
Italy-White-None                 1
Spain-White-Luxury               1
United states-Dessert-Medium     1
Germany-Sparkling-Luxury         1
Chile-Dessert-Premium            1
Chile-Sparkling-Low              1
Chile-Dessert-Medium             1
United states-Dessert-Premium    1
Australia-Dessert-Premium        2
Name: group, dtype: int64


In [ ]:
winedata_df = winedata_df.groupby('group').filter(lambda x : len(x)>50)

In [ ]:
show_snapshot_values('group',True,10)

Nr of rows: 24300
Nr of columns: 16
_____________________________________
 
Total Nr of null values found: 0
_____________________________________
 
Total Nr of Duplicates found: 0
_____________________________________
 
   Value Counts for group
South africa-Rose-Low         51
Italy-Dessert-Premium         51
Australia-White-Premium       54
United states-White-Luxury    55
Portugal-Sparkling-Medium     56
Australia-White-Low           59
Austria-Sparkling-Medium      65
Portugal-White-Premium        65
Argentina-White-Medium        66
United states-Red-Premium     67
Name: group, dtype: int64


# **7.Saving Clean Data**

In [ ]:
## Clean dataframe and make local copy ##
clean_df = winedata_df
### clean_db saved as pickle
clean_df.to_pickle('/content/drive/My Drive/Colab Notebooks/winedata_df_after_data_wrangling.pkl')